![SNOTEL](Images/snotel.jpg)

# Retrieve and Analyze Snotel data for a watershed of interest
Authors: Irene Garousi-Nejad (igarousi@cuahsi.org), Ryan Johnson (ryan.c.johnson@utah.edu)
Last updated: January 9th, 2025

This notebook support the CIROH HydroLearn module Introduction to Seasonal Snow Observations, Modeling, and Analysis, demonstrating how to access NRCS SNOTEL data, in particular,  snow-water-equivalent. 

Supplementary Code:

To simplify this notebook, we developed and import several helper functions from the supporting_scripts folder to streamline data retrieval, data processing, and visualization. 
We encourage advanced users to explore all of the helper scripts in teh supporting_scripts folder.
For example, while this exercise focuses on a river basin in California, there is a script for retrieving SNOTEL observational data for sites located in other states utilizing the traditional site identification number.

# 1. Create Map for Watershed for USGS Station ID
The following code uses the pynhd and folium packages to create an interactive map of a watershed from a USGS gauge ID.

In our exercise, we are tasked with identifying all SNOTEL sites upstream of Hetch Hetchy Reservoir on the Tuolumne River. The user can search for "USGS streamflow Tuolumne River" and serveral locations will pop up. Site [11274790](https://waterdata.usgs.gov/monitoring-location/11274790/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false) is the site of interest for this assessment 

In [ ]:
from pynhd import NLDI, WaterData, NHDPlusHR, GeoConnex
import geopandas as gpd
import pandas as pd
from supporting_scripts import getData, SNOTEL_Analyzer, dataprocessing, mapping
from shapely.geometry import box, Polygon
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

 

Define the watershed outlet using NWIS site id. Create a map object that we'll add layers to.

In [ ]:
nldi = NLDI()
usgs_gage_id = "11274790" # NWIS id for Tuolumne river at the mouth of Hetch Hetchy Reservoir

Collect watershed and reach vectors using the pynhd module.

In [ ]:
nldi = NLDI()

#Getting basin geometry
print('Collecting basins...', end='')
basin = nldi.get_basins(usgs_gage_id)
if not os.path.exists('files'):
    os.makedirs('files')
basin.to_file("files/TuolumneRiverBasin.shp")
print('done')

site_feature = nldi.getfeature_byid("nwissite", f"USGS-{usgs_gage_id}")
upstream_network = nldi.navigate_byid(
    "nwissite", f"USGS-{usgs_gage_id}", "upstreamMain", "flowlines", distance=9999
)

Create and interactive map to display the watershed.

In [ ]:
# create map
mapping.basin_mapping(basin, site_feature)

![Basin](Images/basin.png)

# 1. Identify SNOTEL sites within a spatial domain

Here, we will read a geojson file from the web containing the geospatial information for all SNOTEL sites. 
Once loaded, the script will use the basin geometry (in the form of a polygon in the GeoDataFrame) to clip all SNOTEL sites located within the boundaries of the basin.

In [ ]:
# Create geodataframe of all stations
all_stations_gdf = gpd.read_file('https://raw.githubusercontent.com/egagli/snotel_ccss_stations/main/all_stations.geojson').set_index('code')
all_stations_gdf = all_stations_gdf[all_stations_gdf['csvData']==True]

# Use the polygon geometry to select snotel sites that are within the domain
gdf_in_bbox = all_stations_gdf[all_stations_gdf.geometry.within(basin.geometry[0])]

#reset index to have siteid as a column
gdf_in_bbox.reset_index(drop=False, inplace=True)

#make begin and end date a str
gdf_in_bbox['beginDate'] = [datetime.datetime.strftime(gdf_in_bbox['beginDate'][i], "%Y-%m-%d") for i in np.arange(0,len(gdf_in_bbox),1)]
gdf_in_bbox['endDate'] = [datetime.datetime.strftime(gdf_in_bbox['endDate'][i], "%Y-%m-%d") for i in np.arange(0,len(gdf_in_bbox),1)]
gdf_in_bbox

Add the SNOTEL monitoring station locations to the map

In [ ]:
mapping.snotel_mapping(gdf_in_bbox, basin, site_feature)

![basinsnotel](Images/basinsnotel.png)

# 2. Retrieve data for the selected sites

The following uses the getData.py script to download SNOTEL data for the sites within the domain. Inputs to this script include the SNOTEL site name, site ID, and state abbreviation, which are retrieved from the geodataframe above. Additional inputs include the start and end dates, as well as the path to save the outputs.

In [ ]:
# Use the getData module to retrieve data 
OutputFolder = 'files/SNOTEL'
if not os.path.exists(OutputFolder):
    os.makedirs(OutputFolder)

for i in gdf_in_bbox.index:
    print(i)
    getData.getCaliSNOTELData(gdf_in_bbox.name[i], gdf_in_bbox.code[i], gdf_in_bbox.beginDate[i], gdf_in_bbox.endDate[i], OutputFolder)

# 3.  Process the data to enable plotting and analysis

In [ ]:
# Load the data for one site
sites = list(gdf_in_bbox.code)
stateab = 'Ca'
sitedict = dict()

for site in sites:
    sitedict[site] = dataprocessing.processSNOTEL(site, stateab)

sitedict['TUM']

# 4. Make a snow report for WY2019

In [ ]:
WY = 2019
watershed = "Tuolumne"
AOI = 'Above Hetch Hetchy Reservoir'
DOI = '04-01' #must be in MM-DD form
SNOTEL_Analyzer.SNOTELPlots(sitedict, gdf_in_bbox, WY, watershed, AOI,DOI)

# 5. Generate a Basin Snow report

In this section the user will develop a statistical snow analysis based on the mean values of each SNOTEL observation station within the catchment.

In [ ]:
WY = 2019
watershed = "Tuolumne"
AOI = 'Above Hetch Hetchy Reservoir'
DOI = '04-01' #must be in MM-DD form

SNOTEL_Analyzer.catchmentSNOTELAnalysis(sitedict, WY, watershed, AOI, DOI)

In [ ]:
import os
basinname = 'Tuolumne'
output_res = 1000
# Path to the file you want to remove
folder = f"files/ASO/{basinname}/{output_res}M_SWE_parquet"
date = '20190703'
file_path = f"{folder}/ASO_1000M_SWE_{date}.parquet"

# Check if the file exists
if os.path.exists(file_path):
    # Remove the file
    os.remove(file_path)
    print("file removed")